In [1]:
import requests
import json
from langchain.utilities import BashProcess
import os

In [2]:
def get_api_key():
    terminal = BashProcess(return_err_output=True)
    key = terminal.run('cred="$(az account get-access-token | jq -r \'.accessToken\')" && echo $cred')
    key = key.replace(os.linesep, '')
    return key

In [3]:
# Substrate
_ENDPOINT = 'https://httpqas26-frontend-qasazap-prod-dsm02p.qas.binginternal.com/completions'
# _ENDPOINT = '\
# https://gcrgpt4aoai9.openai.azure.com/openai/deployments/gpt-4/completions?api-version=2022-12-01\
# '
api_key = get_api_key()
model = 'dev-ppo'

_max_retries = 3
max_tokens = 10000
temperature = 0.2

In [4]:
def call_llm(prompt):
    global api_key
    global _ENDPOINT
    global _max_retries
    global max_tokens
    global temperature
    global model
    try:
        request = {
            "prompt": prompt,
            "max_tokens": max_tokens,
            "temperature": temperature,
            "top_p": 1,
            "n": 1,
            "stream": False,
            "stop": "\r\n"
        }
        request_str = json.dumps(request)
        body = str.encode(request_str)
        attempt: int = 0
        while True:
            # -- Substrate LLM API
            headers = {
            'Content-Type':'application/json', 
            'Authorization': 'Bearer ' + api_key,
            'X-ModelType': model }

            # headers = {
            # 'Content-Type':'application/json', 
            # 'api-key': api_key}

            response = requests.post(_ENDPOINT, data=body, headers=headers)
            if response.status_code == 200:
                response_json = json.loads(response.text)
            else:
                if response.text == 'upstream request timeout' and attempt < _max_retries:
                    print(f'Warning: {response.text}; retrying.')
                    attempt += 1
                    continue
                elif ('Microsoft.IdentityModel.Tokens.SecurityTokenExpiredException' in response.text or
                        'Microsoft.IdentityModel.Tokens.SecurityTokenInvalidSignatureException' in response.text or
                        'Failed authenticating sti token' in response.text):
                    print('Warning: Failed authenticating to the LLM API; retrieving key and retrying.')
                    api_key = get_api_key()
                    attempt += 1
                    continue
                response_json = { "error": response.text }
                break
            return json.dumps(response_json)
    except Exception as e:
      print(str(e))
      response_json = { "error": str(e) }
      return json.dumps(response_json)
    
# Function to remove <|im_sep|> and <|im_end|> from the output
def remove_im_sep_end(text):
    text = text.replace("<|im_sep|>", "")
    text = text.replace("<|im_end|>", "")
    return text
    
def parsePromptResponse(response):
    response = json.loads(response)
    if 'error' in response:
      return False, json.loads(response['error'])['error']['message']
    return True, remove_im_sep_end(response['choices'][0]['text'])

In [5]:
def prompt(c_code):
    prompt = f"""You are an expert verification engineer experienced in working with verification tools like Boogie and SMT solvers like Z3. \
You are given a C program with one loop and pre-conditions and post-conditions. \
Your task is to convert it into a horn clause problem that can be input to the Z3 SMT solver. Output must be only SMT code and no plain-text.
---
Input C Program:
{c_code}
---
Output SMT Code:
(set-logic HORN)
    """
    return prompt

def prompt2(mode, c_code):
    prompt = f"""You are LoopInvariantGPT, an assistant for automatic software verification. You take a C program with a loop and an assertion as input and you have two modes. \
In first mode, you can translate the verification of C program to SMT2 constraints with invariant as an unknown function such that these constraints can be consumed by Z3. \
They will include an initialization, one or more transition relations, and a property check. In second mode, you act as a translator from C programs to programs in Boogie programming language. \
Given an input program in C, annotated with assertions, your goal is to create a Boogie program annotated with loop invariants which prove that the assertions hold.
Choose simple invariants when possible. \
Please avoid non-linear arithmetic, 2*x is OK but x*y is disallowed. \
If mode is unspecified then output both the SMT formula and the annotated boogie program.

---
Input:
SMT
int main() {{
int x;
int y;
// pre-conditions
(x = 1);
(y = 0);
// loop body
while ((y < 100000)) {{
{{
(x = (x + y));
(y = (y + 1));
}}
}}
// post-condition
assert( (x >= y) );
}}

LoopInvariantGPT:
(set-logic HORN)
(declare-fun inv (Int Int) Bool)
; Initial condition
(assert (forall ((x Int)(y Int)) (=> ( and (= x 1) (= y 0)) (inv x y))))
; Transition relation
(assert (forall ((x Int)(y Int)(x1 Int)(y1 Int)) (=> (and (inv x y) (< y 100000) (= x1 (+ x y)) (= y1 (+ y 1)) ) (inv x1 y1))))
; Property
(assert (forall ((x Int)(y Int)) (=> (and (inv x y)(not (< y 100000))) (>= x y) )))
(check-sat)
(get-model)

Input:
Boogie
int main() {{
// variable declarations
int x;
int y;
// pre-conditions
assume((x >= 0));
assume((x <= 2));
assume((y <= 2));
assume((y >= 0));
// loop body
while (unknown()) {{
{{
(x = (x + 2));
(y = (y + 2));
}}
}}
// post-condition
if ( (y == 0) )
assert( (x != 4) );
}}

LoopInvariantGPT:
procedure main()
{{
var x: int;
var y: int;
var nondet: bool;
// pre-conditions
assume(0 <= x);
assume(x <= 2);
assume(y <= 2);
assume(y >= 0);
// loop body
havoc nondet;
while (nondet)
invariant x-y <= 2 && y - x <= 2;
{{
havoc nondet;
x := x + 2;
y := y + 2;
}}
// post-condition
if(y==0) {{
assert(x != 4);
}}
}}

Input:
{mode}
{c_code}

LoopInvariantGPT:
    """
    return prompt


In [8]:
res = call_llm(prompt2("SMT","""
int main()
{
    int x = 0;
    int y, z;

    while(x < 5) {
       x += 1;
       if( z <= y) {
          y = z;
       }
    }

    assert (z >= y);
}
"""))
print(parsePromptResponse(res)[1])

 (set-logic HORN)
     (declare-fun inv (Int Int Int) Bool)
     ; Initial condition
     (assert (forall ((x Int)(y Int)(z Int)) (=> (and (= x 0)) (inv x y z))))
     ; Transition relation
     (assert (forall ((x Int)(y Int)(z Int)(x1 Int)(y1 Int)(z1 Int)) (=> (and (inv x y z) (< x 5) (= x1 (+ x 1)) (= z1 z) (ite (<= z y) (= y1 z) (= y1 y))) (inv x1 y1 z1))))
     ; Property
     (assert (forall ((x Int)(y Int)(z Int)) (=> (and (inv x y z)(not (< x 5))) (>= z y) )))
     (check-sat)
     (get-model)


In [ ]:
def convert_file(num):
    with open(f"bench/c_benchmark/{num}.c") as f:
        print("-----------------------------------")
        # Read the file f
        c_code = f.read()
        input_prompt = prompt(c_code)
        print(input_prompt)
        llm_response = call_llm(input_prompt)
        print(llm_response)
        output = parsePromptResponse(llm_response)[1]
        # output = "(set-logic HORN)\n" + output
        print(output)
        print("-----------------------------------")

        with open(f"bench/z3_output/{num}.smt2", "w") as f:
            f.write(output)

In [ ]:
convert_file(3)

In [9]:
for i in range(1, 133):
    with open(f"bench/c_benchmark/{i}.c") as f:
        # print("-----------------------------------")
        # Read the file f
        c_code = f.read()
        input_prompt = prompt2("SMT", c_code)
        # print(input_prompt)
        llm_response = call_llm(input_prompt)
        if llm_response is None: continue
        output = parsePromptResponse(llm_response)[1]
        # output = "(set-logic HORN)\n" + output
        # print(output)
        # print("-----------------------------------")
        print(i)
        # x = input()
        # if x == "q": break

        with open(f"bench/z3_output/{i}.smt2", "w") as f:
            f.write(output)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
